In [3]:

import nltk
import numpy as np
import pandas as pd
import re
import string
import json
from nltk.corpus import reuters, stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Download NLTK resources
nltk.download('reuters')
nltk.download('stopwords')

# Step 1: Tokenization and Preprocessing
class Tokenizer:
    def __init__(self, stop_words=None):
        self.stop_words = stop_words or set(stopwords.words('english'))
    
    def tokenize(self, text):
        text = text.lower()
        text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
        text = re.sub(r"\d+", " ", text)
        tokens = text.split()
        return [word for word in tokens if word not in self.stop_words]

# Custom stopwords
custom_stopwords = set(stopwords.words("english"))
custom_stopwords.update({"mln", "lt", "pct", "dlrs", "vs", "000", "said", "year", "loss", "profit", "net"})

# Load Reuters dataset
documents = reuters.fileids()
texts = [reuters.raw(doc_id) for doc_id in documents]
labels = [reuters.categories(doc_id) for doc_id in documents]

tokenizer = Tokenizer(stop_words=custom_stopwords)
tokenized_texts = [tokenizer.tokenize(text) for text in texts]

# Step 2: Convert texts to Bag-of-Words representation
vectorizer = CountVectorizer(max_features=5000, stop_words=list(custom_stopwords))
X_bow = vectorizer.fit_transform([" ".join(tokens) for tokens in tokenized_texts])

# Step 3: LDA Topic Modeling
n_topics = 15
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
X_lda = lda.fit_transform(X_bow)

# Display top words in each topic
print("LDA Topics:")
for idx, topic in enumerate(lda.components_):
    print(f"Topic #{idx + 1}:")
    print(" ".join([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]]))

# Step 4: Load GloVe Embeddings
def load_glove_embeddings(glove_path, embedding_dim, vocabulary):
    embeddings_index = {}
    with open(glove_path, "r", encoding="utf-8") as file:
        for line in file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype="float32")
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((len(vocabulary) + 1, embedding_dim))
    for word, idx in vocabulary.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
    return embedding_matrix

glove_path = "glove.6B.100d.txt"  # Update path accordingly
embedding_dim = 100
vocabulary = {word: idx for idx, word in enumerate(vectorizer.get_feature_names_out())}
embedding_matrix = load_glove_embeddings(glove_path, embedding_dim, vocabulary)

# Step 5: Combine GloVe Embeddings with LDA Features
X_glove = X_bow.toarray()
X_combined = np.hstack((X_glove, X_lda))

# Step 6: One-Hot Encode Labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)

# Step 7: Train/Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Step 8: Define Focal Loss Function
import tensorflow as tf

def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.keras.backend.clip(y_pred, epsilon, 1. - epsilon)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -alpha * tf.math.pow(1. - pt, gamma) * tf.math.log(pt)
        return tf.reduce_mean(loss)
    return focal_loss_fixed

# Step 9: Build the Model
model = Sequential([
    Dense(512, activation="relu", input_shape=(X_train.shape[1],), kernel_initializer=he_normal()),
    BatchNormalization(),
    Dropout(0.3),
    Dense(256, activation="relu", kernel_initializer=he_normal()),
    BatchNormalization(),
    Dropout(0.3),
    Dense(y_train.shape[1], activation="sigmoid", kernel_initializer=he_normal())
])

# Compile the Model
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=["accuracy"]
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Train the Model
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Predict on Test Set
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_binary, zero_division=0))

# Overall Accuracy
overall_accuracy = accuracy_score(y_test, y_pred_binary)
print(f"\nOverall Test Accuracy: {overall_accuracy}")

# Save the Model
model.save("glove_lda_dense_model.h5")
print("Model saved as 'glove_lda_dense_model.h5'")

[nltk_data] Downloading package reuters to
[nltk_data]     /Users/rebeccavannostrand/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rebeccavannostrand/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LDA Topics:
Topic #1:
usda production agriculture last corn grain ec sugar wheat tonnes
Topic #2:
also dollar market agreement told countries japanese japan would trade
Topic #3:
sales reported expects per gold first share earnings company quarter
Topic #4:
bid inc would stake stock share group company offer shares
Topic #5:
market interest billion week money banks fed rates bank rate
Topic #6:
barrels energy bpd price production opec gas crude prices oil
Topic #7:
tax would new ltd group last profits francs company billion
Topic #8:
nine shrs avg oper inc note revs qtr shr cts
Topic #9:
dealers today money futures one brazil coffee market bank stg
Topic #10:
deficit march surplus rise december fell rose february january billion
Topic #11:
state court workers two would port strike south spokesman union
Topic #12:
rise billion economy bank yen foreign economic government dollar growth
Topic #13:
share sale common co unit shares stock corp company inc
Topic #14:
iranian american federal 

/Users/rebeccavannostrand/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0910 - loss: 0.1078 - val_accuracy: 0.3946 - val_loss: 0.0333 - learning_rate: 5.0000e-04
Epoch 2/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3203 - loss: 0.0461 - val_accuracy: 0.6269 - val_loss: 0.0134 - learning_rate: 5.0000e-04
Epoch 3/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4852 - loss: 0.0161 - val_accuracy: 0.6657 - val_loss: 0.0052 - learning_rate: 5.0000e-04
Epoch 4/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5713 - loss: 0.0069 - val_accuracy: 0.6813 - val_loss: 0.0036 - learning_rate: 5.0000e-04
Epoch 5/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6317 - loss: 0.0044 - val_accuracy: 0.6987 - val_loss: 0.0030 - learning_rate: 5.0000e-04
Epoch 6/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6756 - loss: 0.0034 - val_accuracy: 0.7184 - val_loss: 0.0026 - learning_rate: 5.0000e-04
Epoch 7/100
108/108 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.700

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       469
           1       1.00      0.14      0.25         7
           2       1.00      0.50      0.67         6
           3       0.86      0.60      0.71        20
           4       1.00      0.40      0.57        15
           5       0.00      0.00      0.00         2
           6       0.92      0.71      0.80        17
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       1.00      0.75      0.86        28
          10       1.00      0.67      0.80         9
          11       0.00      0.00      0.00         0
          12       0.84      0.51      0.64        53
          13       0.83      0.45      0.59        11
          14       0.00      0.00      0.00         0
          15       0.90      0.45      0.60        20
          16       0.00      0.00      0.00         1
          17       0.92    